In [62]:

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

import statsmodels.api as sm


from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model,datasets, svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import catboost
from catboost import CatBoostClassifier, Pool, cv

from sklearn.model_selection import GridSearchCV

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train["Sex"].unique()
train['Sex'] = train['Sex'].replace({'male':0,'female':1})
test['Sex'] = test['Sex'].replace({'male':0,'female':1})
train.describe().T
test.describe().T
train.isnull().sum()                                                                             #valores nulos em Age, Cabin, Embarked.
test.isnull().sum()                                                                                   #valores nulos em Age, Fare, Cabin.

train.head(5)
##### titulo no nome
titulo = []
for i in train.index:
    a = train['Name'][i].split(',')[1].split('.')[0].strip()
    titulo.append(a)
train['titulo'] = titulo
titulo = []
for i in test.index:
    a = test['Name'][i].split(',')[1].split('.')[0].strip()
    titulo.append(a)
test['titulo'] = titulo
train['titulo'].unique()
test['titulo'].unique()
train['titulo'].value_counts()
test['titulo'].value_counts()
### correlação para notar como preencher os capos faltantes que são: Age, Fare, Cabin e Embarked.

###### Comparando do DataFrames parece ser justo completar a idade pelo titulo carregado no nome. as medias de idades por titulos aparentam graficamente serem parecidadas.
train.groupby(by='titulo').mean()['Age']
test.groupby(by='titulo').mean()['Age']                                             #Ms teve NaN em Age. será usado entao e media de sua pclass

test.loc[test.titulo=="Ms"]
test.groupby(by='Pclass').mean()['Age']
test['Age'][test.loc[test.titulo=="Ms"].index] = test.groupby(by='Pclass').mean()['Age'][3].round()
test.loc[test.Fare.isna()]
test['Fare'][test.loc[test.Fare.isna()].index] = test.groupby(by='Pclass').mean()['Fare'][3].round()
a = test.groupby(by='titulo').mean()['Age']
for i in test.index:
    if pd.isnull(test.Age[i]):
        test.Age[i] = test.groupby(by='titulo').mean()['Age'][test.titulo[i]].round()
    else:
        continue
a = train.groupby(by='titulo').mean()['Age']
for i in train.index:
    if pd.isnull(train.Age[i]):
        train.Age[i] = train.groupby(by='titulo').mean()['Age'][train.titulo[i]].round()
    else:
        continue
test.head(5)
train.drop(['Cabin','Ticket'], axis=1,inplace=True)
test.drop(['Cabin','Ticket'], axis=1,inplace=True)
for i in train.index:
    if train['titulo'][i] in ['Mr', 'Mrs', 'Miss', 'Master']:
        continue
    else:
        train['titulo'][i] = 'outro'
        
for i in test.index:
    if test['titulo'][i] in ['Mr', 'Mrs', 'Miss', 'Master']:
        continue
    else:
        test['titulo'][i] = 'outro'
train['Embarked'].value_counts()
test['Embarked'].value_counts()
train['titulo'].value_counts()
test['titulo'].value_counts()
test
train

C:\Users\alexf\AppData\Local\Temp\ipykernel_11460\1666532921.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'][test.loc[test.titulo=="Ms"].index] = test.groupby(by='Pclass').mean()['Age'][3].round()
C:\Users\alexf\AppData\Local\Temp\ipykernel_11460\1666532921.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Fare'][test.loc[test.Fare.isna()].index] = test.groupby(by='Pclass').mean()['Fare'][3].round()
C:\Users\alexf\AppData\Local\Temp\ipykernel_11460\1666532921.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,titulo
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,13.0000,S,outro
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,30.0000,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,22.0,1,2,23.4500,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,30.0000,C,Mr


In [63]:
#### One_hot_encoder
onehotencoder = OneHotEncoder()

SyntaxError: invalid syntax (2456085548.py, line 2)

In [68]:
###### train
X11 = onehotencoder.fit_transform(train.Embarked.values.reshape(-1,1)).toarray()
range11 = X11.shape[1]
range11
X11

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [ ]:
X12 =  onehotencoder.fit_transform(train.titulo.values.reshape(-1,1)).toarray()
range12 = X12.shape[1]


In [ ]:
dfOneHot11 = pd.DataFrame(X11, columns=['Embarked_'+str(int(i)) for i in range(range11)])
dfOneHot12 = pd.DataFrame(X12, columns=['titulo_'+str(int(i)) for i in range(range12)])

In [ ]:
train = pd.concat([train, dfOneHot11,dfOneHot12], axis=1)

SyntaxError: invalid syntax (839791075.py, line 1)

In [ ]:
###### test
X21 = onehotencoder.fit_transform(test.Embarked.values.reshape(-1,1)).toarray()
range21 = X21.shape[1]



In [ ]:
X22 =  onehotencoder.fit_transform(test.titulo.values.reshape(-1,1)).toarray()
range22 = X22.shape[1]
dfOneHot21 = pd.DataFrame(X21, columns=['Embarked_'+str(int(i)) for i in range(range21)-1])
dfOneHot22 = pd.DataFrame(X22, columns=['titulo_'+str(int(i)) for i in range(range22)])

In [ ]:
test = pd.concat([test, dfOneHot21,dfOneHot22], axis=1)
##### drop apos criacao de colunas para cada variavel categorica
test.drop(['Name','Embarked','titulo'],axis=1,inplace=True)
train.drop(['Name','Embarked','titulo'],axis=1,inplace=True)


In [ ]:
test.columns.to_list()

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'titulo']

In [ ]:
train.columns.to_list()

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'titulo',
 'Embarked_0',
 'Embarked_1',
 'Embarked_2',
 'Embarked_3',
 'titulo_0',
 'titulo_1',
 'titulo_2',
 'titulo_3',
 'titulo_4']

In [ ]:
train["Embarked_0"].value_counts()

0.0    723
1.0    168
Name: Embarked_0, dtype: int64

In [ ]:
train["Embarked_1"].value_counts()

0.0    814
1.0     77
Name: Embarked_1, dtype: int64

In [ ]:
train["Embarked_2"].value_counts()


1.0    644
0.0    247
Name: Embarked_2, dtype: int64

In [ ]:
train["Embarked_3"].value_counts()


0.0    889
1.0      2
Name: Embarked_3, dtype: int64

In [ ]:
train["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
test["Embarked"].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64